# 🎯 Outlier Detection

**บทเรียนที่ 2.5k - การหา Outliers**

---

## 🎯 วัตถุประสงค์

หลังจากเรียนบทนี้แล้ว ผู้เรียนจะสามารถ:
- เข้าใจประเภทของ Outliers
- ใช้วิธีการต่างๆ ในการหา Outliers
- ตัดสินใจว่าจะจัดการ Outliers อย่างไร
- ประเมินผลกระทบของ Outliers ต่อการวิเคราะห์

In [ ]:
# นำเข้า libraries ที่จำเป็น
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette("Set1")

print("🎯 Outlier Detection")
print("=" * 25)

## 📖 ประเภทของ Outliers

### 1. 📊 **Statistical Outliers**
- ค่าที่อยู่นอกช่วงปกติทางสถิติ
- ใช้วิธี IQR, Z-score

### 2. 🎯 **Contextual Outliers**
- ค่าที่ผิดปกติในบริบทเฉพาะ
- ตัวอย่าง: อุณหภูมิ 25°C ในฤดูหนาว

### 3. 🔍 **Global vs Local Outliers**
- **Global**: ผิดปกติเมื่อดูทั้งชุดข้อมูล
- **Local**: ผิดปกติเฉพาะในกลุ่มย่อย

### 4. ⚠️ **Data Entry Errors vs True Outliers**
- **Errors**: ความผิดพลาดในการบันทึก
- **True**: ค่าจริงที่ผิดปกติ

## 📊 สร้างข้อมูลที่มี Outliers

In [ ]:
# สร้างข้อมูล Lending Club ที่มี outliers
np.random.seed(42)
n_normal = 950
n_outliers = 50

# ข้อมูลปกติ
normal_income = np.random.lognormal(13, 0.5, n_normal).astype(int)
normal_loan = normal_income * np.random.uniform(0.2, 0.4, n_normal)
normal_interest = 8 + np.random.normal(0, 3, n_normal)
normal_dti = np.random.uniform(10, 30, n_normal)

# เพิ่ม outliers
# รายได้สูงผิดปกติ
outlier_income = np.random.uniform(2000000, 5000000, n_outliers//5).astype(int)
# เงินกู้สูงผิดปกติ
outlier_loan = np.random.uniform(800000, 1500000, n_outliers//5)
# อัตราดอกเบี้ยสูงผิดปกติ
outlier_interest = np.random.uniform(25, 35, n_outliers//5)
# DTI สูงผิดปกติ
outlier_dti = np.random.uniform(50, 80, n_outliers//5)
# ค่าติดลบ (data entry error)
error_values = np.random.uniform(-50000, -1000, n_outliers//5)

# รวมข้อมูล
all_income = np.concatenate([
    normal_income, 
    outlier_income,
    np.random.choice(normal_income, n_outliers//5),  # เติมให้ครบ
    np.random.choice(normal_income, n_outliers//5),
    np.abs(error_values).astype(int) + 200000  # แปลงให้เป็นบวก
])

all_loan = np.concatenate([
    normal_loan,
    np.random.choice(normal_loan, n_outliers//5),
    outlier_loan,
    np.random.choice(normal_loan, n_outliers//5),
    np.random.choice(normal_loan, n_outliers//5)
])

all_interest = np.concatenate([
    normal_interest,
    np.random.choice(normal_interest, n_outliers//5),
    np.random.choice(normal_interest, n_outliers//5),
    outlier_interest,
    np.random.choice(normal_interest, n_outliers//5)
])

all_dti = np.concatenate([
    normal_dti,
    np.random.choice(normal_dti, n_outliers//5),
    np.random.choice(normal_dti, n_outliers//5),
    np.random.choice(normal_dti, n_outliers//5),
    outlier_dti
])

# สร้าง DataFrame
df_outliers = pd.DataFrame({
    'loan_id': [f'LC{i:06d}' for i in range(1, len(all_income) + 1)],
    'annual_income': all_income,
    'loan_amount': all_loan.astype(int),
    'interest_rate': np.clip(all_interest, 5, 30),  # จำกัดช่วง
    'dti': all_dti
})

print("📊 ข้อมูลที่มี Outliers:")
print(f"ขนาด: {df_outliers.shape}")
print(df_outliers.describe())

## 📊 วิธีการหา Outliers

In [ ]:
def detect_outliers_iqr(data, column_name):
    """
    หา Outliers ด้วยวิธี IQR (Interquartile Range)
    """
    print(f"\n📊 IQR Method: {column_name}")
    print("-" * 40)
    
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    
    # คำนวณขอบเขต
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # หา outliers
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    outlier_indices = outliers.index
    
    print(f"📏 Q1: {Q1:,.2f}")
    print(f"📏 Q3: {Q3:,.2f}")
    print(f"📏 IQR: {IQR:,.2f}")
    print(f"📉 Lower bound: {lower_bound:,.2f}")
    print(f"📈 Upper bound: {upper_bound:,.2f}")
    print(f"🎯 จำนวน Outliers: {len(outliers)} ({len(outliers)/len(data)*100:.1f}%)")
    
    if len(outliers) > 0:
        print(f"📋 ค่า Outliers (5 ตัวแรก): {outliers.head().tolist()}")
    
    return outlier_indices, lower_bound, upper_bound

def detect_outliers_zscore(data, column_name, threshold=3):
    """
    หา Outliers ด้วยวิธี Z-score
    """
    print(f"\n📊 Z-Score Method: {column_name}")
    print("-" * 40)
    
    # คำนวณ Z-score
    z_scores = np.abs(stats.zscore(data))
    
    # หา outliers
    outlier_indices = data[z_scores > threshold].index
    outlier_values = data[z_scores > threshold]
    
    print(f"📏 Mean: {data.mean():,.2f}")
    print(f"📏 Std: {data.std():,.2f}")
    print(f"🎯 Threshold: {threshold} standard deviations")
    print(f"🎯 จำนวน Outliers: {len(outlier_values)} ({len(outlier_values)/len(data)*100:.1f}%)")
    
    if len(outlier_values) > 0:
        print(f"📋 ค่า Outliers (5 ตัวแรก): {outlier_values.head().tolist()}")
        print(f"📋 Z-scores (5 ตัวแรก): {z_scores[outlier_indices].head().tolist()}")
    
    return outlier_indices

def detect_outliers_modified_zscore(data, column_name, threshold=3.5):
    """
    หา Outliers ด้วยวิธี Modified Z-score (ใช้ Median)
    """
    print(f"\n📊 Modified Z-Score Method: {column_name}")
    print("-" * 40)
    
    # คำนวณ Modified Z-score
    median = data.median()
    mad = np.median(np.abs(data - median))  # Median Absolute Deviation
    
    modified_z_scores = 0.6745 * (data - median) / mad
    
    # หา outliers
    outlier_indices = data[np.abs(modified_z_scores) > threshold].index
    outlier_values = data[np.abs(modified_z_scores) > threshold]
    
    print(f"📏 Median: {median:,.2f}")
    print(f"📏 MAD: {mad:,.2f}")
    print(f"🎯 Threshold: {threshold}")
    print(f"🎯 จำนวน Outliers: {len(outlier_values)} ({len(outlier_values)/len(data)*100:.1f}%)")
    
    if len(outlier_values) > 0:
        print(f"📋 ค่า Outliers (5 ตัวแรก): {outlier_values.head().tolist()}")
    
    return outlier_indices

# ทดสอบกับ annual_income
print("🔍 การหา Outliers: Annual Income")
print("=" * 50)

iqr_outliers, lower_bound, upper_bound = detect_outliers_iqr(df_outliers['annual_income'], 'annual_income')
zscore_outliers = detect_outliers_zscore(df_outliers['annual_income'], 'annual_income')
modified_outliers = detect_outliers_modified_zscore(df_outliers['annual_income'], 'annual_income')

## 📚 สรุป

### ✅ สิ่งที่เราได้เรียนรู้:

1. **วิธีการหา Outliers**:
   - IQR Method: เหมาะกับข้อมูลที่ไม่เป็นปกติ
   - Z-Score: เหมาะกับข้อมูลที่เป็นปกติ
   - Modified Z-Score: ทนต่อ outliers มากกว่า

2. **การตีความ Outliers**:
   - พิจารณาบริบททางธุรกิจ
   - แยกแยะ errors กับ true outliers
   - ประเมินผลกระทบต่อการวิเคราะห์

3. **การจัดการ Outliers**:
   - ลบออก (หากเป็น errors)
   - แปลงข้อมูล (log transformation)
   - ใช้วิธีทางสถิติที่ทนต่อ outliers
   - เก็บไว้ (หากเป็นข้อมูลสำคัญ)

### 🚀 บทถัดไป:
เราจะเรียนรู้ **Workshop: Data Profiling ข้อมูล Lending Club**

---
*💡 เคล็ดลับ: Outliers ไม่ใช่ศัตรู - อาจเป็นข้อมูลที่มีค่าที่สุด!*